# Базовые сведения о процессах

У процесса бывают следующие характеристики:

* Идентификатор процесса.  
* Объем оперативной памяти.  
* Стек.  
* Список открытых файлов.  
* Ввод/вывод.  

Посмотрим запущенные на текущий момент процессы:

In [1]:
%%bash

top -b -n 1

top - 20:19:35 up 0 min,  0 users,  load average: 0.52, 0.58, 0.59
Tasks:   4 total,   1 running,   3 sleeping,   0 stopped,   0 zombie
%Cpu(s): 25.7 us, 31.7 sy,  0.0 ni, 41.4 id,  0.0 wa,  1.2 hi,  0.0 si,  0.0 st
KiB Mem :  8288672 total,  3457196 free,  4595000 used,   236476 buff/cache
KiB Swap: 24361084 total, 24286216 free,    74868 used.  3552816 avail Mem 

  PID USER      PR  NI    VIRT    RES    SHR S  %CPU %MEM     TIME+ COMMAND
    1 root      20   0    8892    308    272 S   0.0  0.0   0:00.09 init
    5 root      20   0    8916    232    180 S   0.0  0.0   0:00.01 init
    6 cyril     20   0   13636   1872   1768 S   0.0  0.0   0:00.10 bash
   22 cyril     20   0   15768   1796   1324 R   0.0  0.0   0:00.03 top


Теперь создадим наш Python-процесс и посмотрим на его характеристики средствами операционной системы, после чего остановим его.

In [2]:
%%writefile sleep.py

import os
import time

pid = os.getpid()

while True:
    print(pid, time.time())
    time.sleep(2)

Writing sleep.py


In [3]:
%%bash

python3 sleep.py &

#top -b -n 1 | grep python3
ps aux | head -1; ps axu | grep "sleep.py"
top -b -n 1 | grep python3 | awk '{print $1}' | xargs -r kill -9

USER       PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
cyril       40  0.0  0.0  20396  4128 tty1     R    20:19   0:00 python3 sleep.py
cyril       44  0.0  0.0  12892  1124 tty1     S    20:19   0:00 grep sleep.py


-bash: line 6:    40 Killed                  python3 sleep.py


При своей работе программа выполняет системные вызовы (что логично). Эти вызовы можно отслеживать командой `strace`:

`sudo strace -p <pid>`

Также программа может работать с разными файлами. Посмотреть их можно командой `lsof`:

`lsof -p <pid>`

# Программное создание процессов в Python

Сабж делается вызовом функции `os.fork()`. Посмотрим пример.

In [4]:
%%writefile child.py

import time
import os

pid = os.fork()

#Дочерний процесс
if pid == 0:
    for _ in range(3):
        print(f'child: {os.getpid()}; time: {time.time()}')
        time.sleep(2)
else:
    print(f'parent: {os.getpid()}')
    os.wait()

Writing child.py


In [5]:
%%bash

python3 child.py

child: 68; time: 1630775976.6424167
child: 68; time: 1630775978.6430752
child: 68; time: 1630775980.6441362
parent: 67


Вызов `os.fork()`, как уже сказано, создает дочерний процесс. При этом дочернему процессу передаются все системные ресурсы, с которыми работает родительский процесс - копируются в т.ч. память и файловые дескрипторы. Также по коду можно понять, что функция `os.fork()` в дочерний процесс возвращает 0, а в родительский - `pid` дочернего процесса. А еще по коду можно понять, что вызов `os.wait()` дожидается завершения всех дочерних процессов. 

Если в bash мы хотим понять, какой процесс является родительским, а какой - дочерним, то нам поможет команда `ps -axf`.

Вызов функции `os.fork()` может рассматриваться лишь как иллюстрация создания дочерних процессов. На практике он не очень удобен и используют класс `Process` из модуля `multiprocessing`. Посмотрим, как это выглядит.

In [6]:
%%writefile no_fork.py

from multiprocessing import Process

def f(name: str):
    print(f'Hello {name}')
    
p = Process(target=f, args=('Bob',))
p.start()
p.join()

Writing no_fork.py


In [7]:
%%bash

python3 no_fork.py

Hello Bob


Еще более удобным является альтернативный способ использования класса `multiprocessing.Process`, когда мы создаем класс-наследник от `multiprocessing.Process`, передаем в инициализатор нужные для работы параметры и переопределяем метод `run()`:

In [8]:
%%writefile process_class.py

import sys
from multiprocessing import Process

class PrintProcess(Process):
    def __init__(self, name):
        super().__init__()
        self.name = name
        
    def run(self):
        print(f'hello {self.name}')
        
if __name__ == '__main__':
    p = PrintProcess(sys.argv[1])
    p.start()
    p.join()

Writing process_class.py


In [9]:
%%bash 

python3 process_class.py Cyril

hello Cyril


# Создание потоков в Python

Основное отличие потоков от процессов состоит в том, что потоки выполняются в рамках одного процесса, разделяя его память и другие ресурсы. При этом у каждого потока есть собственный стек. Посмотрим, как создать поток на Python.

In [10]:
%%writefile th.py

from threading import Thread

def f(name: str):
    print(f'Hello {name}!')
    
th = Thread(target=f, args=('Bob',))
th.start()
th.join

Writing th.py


In [11]:
%%bash

python3 th.py

Hello Bob!


Как и процесс, поток можно создать альтернативным способом - при помощи наследования.

In [12]:
%%writefile th_inh.py

import sys
from threading import Thread

class PrintThread(Thread):
    def __init__(self, name):
        super().__init__()
        self.name = name
        
    def run(self):
        print(f'Hello {self.name}')
        
if __name__ == '__main__':
    p = PrintThread(sys.argv[1])
    p.start()
    p.join()

Writing th_inh.py


In [13]:
%%bash

python3 th_inh.py Cyril

Hello Cyril


Также в Python3 есть очень интересная возможность запускать одну функцию с разными аргументами, используя фиксированное число потоков. Допустим, мы хотим многопоточно посчитать квадраты чисел от 0 до 9, используя не более 4 потоков.

In [14]:
%%writefile thread_pool.py

from concurrent.futures import ThreadPoolExecutor, as_completed

def f(a):
    return a * a

with ThreadPoolExecutor(max_workers=4) as pool:
    results = [pool.submit(f, i) for i in range(10)]
    
    for future in as_completed(results):
        print(future.result())

Writing thread_pool.py


In [15]:
%%bash

python3 thread_pool.py

36
64
0
25
9
4
81
1
16
49


# Синхронизация потоков и блокировки

Для потока можно организовать очередь задач. Как только место в этой очереди заканчивается, программа переходит в режим ожидания выполнения текущей задачи и новые задачи в очередь не пускает. На практике организация очереди задач выглядит вот так:

In [18]:
%%writefile queue_example.py

from queue import Queue
from threading import Thread

def worker(q: Queue, n: int):
    while True:
        item = q.get()
        if item is None:
            break
        print(f'process data: {n}, {item}')
        
q = Queue(5)
th1 = Thread(target=worker, args=(q, 1))
th2 = Thread(target=worker, args=(q, 2))
th1.start(); th2.start();

for i in range(50):
    q.put(i)
    
q.put(None); q.put(None);
th1.join(); th2.join();

Overwriting queue_example.py


In [19]:
%%cmd

python queue_example.py

Microsoft Windows [Version 10.0.18362.113]
(c) 2019 Microsoft Corporation. All rights reserved.

]9;8;"USERNAME"\@]9;8;"COMPUTERNAME"\ c:\Worker\Python\PythonMailRu\5.MultithreadedAndAsync
# ]9;12\
]9;8;"USERNAME"\@]9;8;"COMPUTERNAME"\ c:\Worker\Python\PythonMailRu\5.MultithreadedAndAsync
# ]9;12\python queue_example.py
process data: 2, 0
process data: 2, 1
process data: 2, 2
process data: 2, 3
process data: 2, 4
process data: 2, 5
process data: 2, 6
process data: 2, 7
process data: 2, 8
process data: 2, 9
process data: 2, 10
process data: 2, 11
process data: 2, 12
process data: 2, 13
process data: 2, 14
process data: 2, 15
process data: 2, 16
process data: 2, 17
process data: 2, 18
process data: 2, 19
process data: 1, 20
process data: 1, 21
process data: 1, 22
process data: 1, 23
process data: 1, 24
process data: 1, 25
process data: 1, 26
process data: 1, 27
process data: 1, 28
process data: 1, 29
process data: 1, 30
process data: 1, 31
process data: 1, 32
process data: 1,